In [ ]:
# This file shows how to save a keras model to tensorflow pb file 
# and how to use tensorflow to reload the model and inferece by the model 

from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
import tensorflow as tf
import numpy as np
np.random.seed(0)

import matplotlib.pyplot as plt

from keras import backend as K

# parameter ==========================
wkdir = './checkpoints'
pb_filename = 'model.pb'
pbtxt_filename = 'model.pbtxt'

In [ ]:
# build a keras model ================
# MNIST CNN

mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# model = tf.keras.models.Sequential([
#   tf.keras.layers.Flatten(input_shape=(28, 28)),
#   tf.keras.layers.Dense(512, activation=tf.nn.relu),
#   # tf.keras.layers.Dropout(0.2),
#   tf.keras.layers.Dense(10, activation=tf.nn.softmax)
# ])

# Dense Model
# model = Sequential()
# model.add(Flatten(input_shape=(28, 28)))
# model.add(Dense(units = 512, activation ='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(10, activation=tf.nn.softmax))

# CNN Model
# image data format compability
# input image dimensions
img_rows, img_cols = 28, 28

if K.image_data_format() == 'channels_first':
    print('channels_first')
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    print('channel_last')
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

# The model (CNN)
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

print("model IO")
print(model.inputs)
print(model.outputs)

model.compile(optimizer='Adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=1)
model.evaluate(x_test, y_test)
predictions_1 = model.predict(x_test[0:10000, :, :, :])

In [ ]:
# build a keras model ================
# Some Random Model
# x = np.vstack((np.random.rand(1000,10),-np.random.rand(1000,10)))
# y = np.vstack((np.ones((1000,1)),np.zeros((1000,1))))
# print(x.shape)
# print(y.shape)

# model = Sequential()
# model.add(Dense(units = 32, input_shape=(10,), activation ='relu'))
# model.add(Dense(units = 16, activation ='relu'))
# model.add(Dense(units = 1, activation ='sigmoid'))

# model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['binary_accuracy'])
# model.fit(x = x, y=y, epochs = 2, validation_split=0.2) 

In [ ]:
# save model to pb ====================
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph

# save keras model as tf pb files ===============
from keras import backend as K
frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])

tf.train.write_graph(frozen_graph, wkdir, pb_filename, as_text=False)
tf.train.write_graph(frozen_graph, wkdir, pbtxt_filename, as_text=True)

In [ ]:
# # load & inference the model ==================

from tensorflow.python.platform import gfile
with tf.Session() as sess:
    # load model from pb file
    with gfile.FastGFile(wkdir+'/'+pb_filename,'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        sess.graph.as_default()
        g_in = tf.import_graph_def(graph_def)
    # write to tensorboard (check tensorboard for each op names)
    writer = tf.summary.FileWriter(wkdir+'/log/')
    writer.add_graph(sess.graph)
    writer.flush()
    writer.close()
    # print all operation names 
#     print('\n===== ouptut operation names =====\n')
#     for op in sess.graph.get_operations():
#       print(op)
    # inference by the model (op name must comes with :0 to specify the index of its output)
    # tensor_output = sess.graph.get_tensor_by_name('import/dense_3/Sigmoid:0')
    # tensor_input = sess.graph.get_tensor_by_name('import/dense_1_input:0')
    tensor_output = sess.graph.get_tensor_by_name('import/dense_2/Softmax:0')
    tensor_input = sess.graph.get_tensor_by_name('import/conv2d_1_input:0')
    predictions = sess.run(tensor_output, {tensor_input: x_test[0:10,:,:,:]})
    print('\n===== output predicted results =====\n')
    print(predictions)
    print(np.argmax(predictions[8]))

In [ ]:
# visualize the prediction
prediction_num = 8
prediction = model.predict(x_test[0:10, :, :, :])
plt.imshow(x_test[prediction_num,:,:,0])
np.argmax(predictions[prediction_num])

In [ ]:
# verify the two system are the same
from tensorflow.python.platform import gfile
with tf.Session() as sess:
    # load model from pb file
    with gfile.FastGFile(wkdir+'/'+pb_filename,'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        sess.graph.as_default()
        g_in = tf.import_graph_def(graph_def)
    # write to tensorboard (check tensorboard for each op names)
    writer = tf.summary.FileWriter(wkdir+'/log/')
    writer.add_graph(sess.graph)
    writer.flush()
    writer.close()
    # print all operation names 
#     print('\n===== ouptut operation names =====\n')
#     for op in sess.graph.get_operations():
#       print(op)
    # inference by the model (op name must comes with :0 to specify the index of its output)
    # tensor_output = sess.graph.get_tensor_by_name('import/dense_3/Sigmoid:0')
    # tensor_input = sess.graph.get_tensor_by_name('import/dense_1_input:0')
    tensor_output = sess.graph.get_tensor_by_name('import/dense_2/Softmax:0')
    tensor_input = sess.graph.get_tensor_by_name('import/conv2d_1_input:0')
    
    # predict from the saved weights
    print('\n===== output predicted results =====\n')
    for i in range(10000):
        # prediction after the execution
        predictions_2 = sess.run(tensor_output, {tensor_input: x_test[i:i+1,:,:,:]})
        label_2 = np.argmax(predictions_2[0])

        label_1 = np.argmax(predictions_1[i])
        if (label_1 != label_2):
            print("label_1: {}; label_2: {}".format(label_1, label_2))
    print('\n===== DONE=====\n')